# Fisher Information

a collection of methods in order to calculate the fisher matrix

In [1]:
#all imports
import classy as cl
from classy import Class
import numpy as np
import matplotlib.pyplot as plt
import pickle
import math
import copy

pi = np.pi

#imports for volume methods
from scipy.integrate import quad
from scipy import integrate
import time
from scipy.interpolate import interp1d

In [2]:
#constants for the volume methods

hbar=6.58*10**(-16)  # eV s units
eq=0.302 # electric charge
alpha=eq**2/(4*np.pi)
Mpl=2.435*10**(18)
c=2.998*10**5 #km/s
ccm=2.998*10**(10) #cm/s

h=.678
OmegaL=0.692
OmegaM=0.308
H100=100/(3.086*10**(19.))*hbar  # 100 km/s/Mpc *(1 Mpc/3.086*10^19 km) * hbar(s eV) -> eV
rhoc = 3*((Mpl*10**(9))**2)*(h**2)*(H100**2) #in eV^(4), need to convert Mpl in GeV to eV
kB= 8.617*10**(-5) #boltzmann's constant in eV / K
Tgamma=2.725*kB
rhogamma= 2*(np.pi**2)*(Tgamma**4)/(30.)
OmegaR=rhogamma/rhoc

In [3]:
def quick_Pk_zbins(VolGpc, z_bins, k_MAX=0.7, add={'N_ur':3.046}):
    z_string=''
    for elem in z_bins[:-1]:
        z_string+=str(elem)+','
    z_string+=str(z_bins[-1])
    k_MIN_Gpc=2*np.pi/float(VolGpc)**(1/3.)
    k_MIN = k_MIN_Gpc*10**(-3)
    
    k_array = np.arange(k_MIN, k_MAX, k_MIN)
    #define a dictionary with the parameters of desired cosmology
    params = {
            'output': 'tCl pCl lCl mPk',
            'l_max_scalars': 5000,
            'lensing': 'yes',
            'A_s': 2.1e-9,
            'n_s': 0.9667,
            '100*theta_s': 1.04112,
            'omega_b':0.02230,
            'omega_cdm': 0.1188,
            'tau_reio': 0.066,
            'N_ur': 3.046
    }
    for key in add.keys():
        params[key] = add[key]
    params['z_pk'] = z_string
    Pk = {}
    cosmo = Class()
    cosmo.set(params)
    cosmo.compute()
    h = cosmo.h()
    #there's no xrange in Python3
    for z_k in z_bins:
        Pk[z_k] = np.zeros(len(k_array))
        for i in range(len(k_array)):
            Pk[z_k][i] = cosmo.pk(h*k_array[i], z_k)*h**3
    cosmo.struct_cleanup()
    cosmo.empty()
    return Pk,k_array,params

In [4]:
def derivative(parameters, step_sizes, VolGpc, z_bins, k_MAX):
    """
    Parameters
    ---------
    parameters : dictionary
        contains the values of the parameters that will be used
        to calculate the derivative
    step_sizes : dictionary
        contains the sizes of the steps that will be taken when
        computing the different cosmos.
    VolGpc : number
        the value for the volume of the galaxy survey which will
        be used as a parameter for quick_Pk_zbins
    z_bins : list
        the different bins of the redshift which will be used as
        a parameter for quick_Pk_zbins
    k_MAX : number
        the maximum k value which will be used to calculate the
        power spectrum in quick_Pk_zbins
                
    Returns
    ------
    partials : dictionary
        contains a dictionary of dictionaries where the first
        key value represents the redshift 'z' and then the
        second key value represents the different parameters
        which were stepped over for the derivative; the values
        of the inner dictionary are arrays which are the
        derivatives for the specified ks
    k_array : array
        contains the different k values for the power spectrum
                
    Function used to calculate the numerical derivative:
                f'(param) = (f(param+h)-f(param-h))/(2h)
        where h is the step size of some parameter param
    """
    
    partials = {}
    #NOTE: This may be slow due to the z_bins being the outer loop
    for z in z_bins:
        add = {} #a parameter for quick_Pk_zbins
        partials[z] = {}
        for elem in step_sizes:
            step = step_sizes[elem]
            add[elem] = parameters[elem] + step
            Pk_1, k_array, params_1 = quick_Pk_zbins(VolGpc, z_bins, k_MAX, add)

            add[elem] = parameters[elem] - step
            Pk_2, k_array, params_2 = quick_Pk_zbins(VolGpc, z_bins, k_MAX, add)
            
            add = {} #reset the dictionary
            
            partials[z][elem] = (Pk_1[z] - Pk_2[z])/(2*step)
    return partials, k_array

With the derivative function, I want to run it only once as long as the step sizes for the parameters doesn't change. I can do this by pickling the resulting output and then using that in other functions.

The equation used to calculate the fisher matrix is given below:

$$F_{ij} = \int_{k_{min}}^{k_{max}} \frac{dk k^2}{(2\pi)^2} \frac{\partial P(k)}{\partial \theta_i} \frac{\partial P(k)}{\partial \theta_j} V_{eff}(k)$$

where $$V_{eff}(k) \approx \left[ \frac{1}{P_g(k) + 1/\bar{n_g}} \right]^2 V$$

In [5]:
def fisher_information(step_sizes, derivatives, Pk, k_array, z, galaxy_density, VolGpc, b):
    """
    Parameters
    ---------
    step_sizes : dictionary
        contains the different parameters that we will be
        concerned with
    derivatives : dictionary
        contains the derivatives of the power spectrum with
        respect to the parameters in step_sizes
    Pk : dictionary
        contains the power spectrum for the different
        redshift values
    k_array : list
        all the associated k values in the power spectrum
        derivatives
    z : number
        the redshift value we are concerned with
    galaxy_density : number
        the average number of galaxies per unit volume
    VolGpc : number
        the volume of the survey
                
    Returns
    ------
    fisher : dictionary
        contains more dictionaries inside where each one will
        represent a row in 2D matrix, due to the symmetry of
        matrix, the order of calling row or col should result
        in the same value being returned
    """
    
    fisher = {} #initialize the fisher matrix (dictionary) to return
    k_min = k_array[0]
    for row in step_sizes:
        fisher[row] = {}
        for col in step_sizes:
            value = 0
            for k in range(len(k_array)):
                V_eff = (Pk[z][k]*(b**2) + 1/galaxy_density)**(-2.)*VolGpc*(10**9)
                value += k_min*((k_array[k])**2)/((2*pi)**2)*derivatives[z][row][k]*derivatives[z][col][k]*V_eff
            fisher[row][col] = value
    return fisher

In [6]:
def matrix_maker(pseudo_matrix):
    """
    Parameters
    ---------
    pseudo_matrix : dictionary
        contains keys indicating the index of the row and the
        values are nested dictionaries with their keys
        indicating the index of the column and their values
        being the actual value in the matrix
        
    Returns
    ------
    actual_matrix : matrix
        a numpy matrix
    indices : list
        contains the order of each index of the matrix    
    """
    
    size = len(pseudo_matrix)
    matrix = np.zeros((size, size))
    
    indices = []
    for row, i in zip(pseudo_matrix, range(size)):
        indices.append(row)
        for col, j in zip(pseudo_matrix[row], range(size)):
            matrix[i][j] = pseudo_matrix[row][col]
    actual_matrix = np.matrix(matrix)
    return actual_matrix, indices

In [7]:
def find_sigma(fisher, indices):
    """
    Parameters
    ---------
    fisher : matrix
        a numpy matrix
    indices : list
        contains the order of each index of the matrix
    
    Returns
    ------
    sigma : dictionary
        contains the list of all the parameters and their
        associated error
    """
    
    sigma = {}
    print('Determinant : ' + str(np.linalg.det(fisher)))
    cov = fisher.I
    
    for index, i in zip(indices, range(len(fisher))):
        sigma[index] = str((cov.item((i, i)))**(1/2))
        print('Error for ' + index + ' is : ' + sigma[index])
    return sigma

# Stochastic Bias

In the following section, I will include a stochastic bias which modifies the power spectrum output by classy

$$P_g(k) = b^2 P_{CLASS}(k)$$

To include a new parameter b, all the other derivatives will be multiplied by $b^2$. Analytically, I can find the derivative with respect to b, due to the simplicity of the polynomial in b.

$$\frac{\partial P_g(k)}{\partial b} = 2b P_{CLASS}(k)$$

In [8]:
def add_bias_derivative(parameters, derivatives, b, VolGpc, z_bins, k_MAX):
    """
    Parameters
    ---------
    parameters : dictionary
        contains the values of the parameters that will be used
        to calculate the derivative
    derivatives : dictionary
        the derivatives dictionary before considering the bias
    b : number
        the bias
    VolGpc : number
        the value for the volume of the galaxy survey which will
        be used as a parameter for quick_Pk_zbins
    z_bins : list
        the different bins of the redshift which will be used as
        a parameter for quick_Pk_zbins
    k_MAX : number
        the maximum k value which will be used to calculate the
        power spectrum in quick_Pk_zbins
    
    Returns
    -------
    derivatives : dictionary
        an updated dictionary with the bias included
    """
    db = copy.deepcopy(derivatives)
    for z in db:
        for param in db[z]:
            db[z][param] *= (b**2)
    Pk, k_array, params = quick_Pk_zbins(VolGpc, z_bins, k_MAX)
    for z in z_bins:
        derivative_b = 2*b*Pk[z]
        db[z]['b'] = derivative_b
    return db

# Capsulation Methods

In the following methods, I will combine previous methods into a single method, so that I can make a single call with all the parameters and get the fisher matrix and errors without having to go through every step.

In [23]:
def fisher_sim_all(parameters, step_sizes, vol_Gpc, num_gal, z, k_max):
    derivatives, k_array = derivative(parameters, step_sizes, vol_Gpc, z_bins, k_max)
    Pk, k_array, params = quick_Pk_zbins(vol_Gpc, z_bins, k_max)
    
    #simple so bias b = 1
    b = 1
    galaxy_density = num_gal/(vol_Gpc*(10**9))
    fisher = fisher_information(step_sizes, derivatives, Pk, k_array, z, galaxy_density, vol_Gpc, b)
    matrix, indices = matrix_maker(fisher)
    print(matrix)
    sigma = find_sigma(matrix, indices)
    
    return fisher, derivatives, sigma
    

In [24]:
def fisher_bias_all(parameters, step_sizes, vol_Gpc, num_gal, z, k_max, b):
    derivatives, k_array = derivative(parameters, step_sizes, vol_Gpc, z_bins, k_max)
    Pk, k_array, params = quick_Pk_zbins(vol_Gpc, z_bins, k_max)
    
    ss = copy.deepcopy(step_sizes)
    derivatives_b = add_bias_derivative(parameters, derivatives, b, vol_Gpc, z_bins, k_max)
    ss['b'] = 0
    
    galaxy_density = num_gal/(vol_Gpc*(10**9))
    fisher = fisher_information(ss, derivatives_b, Pk, k_array, z, galaxy_density, vol_Gpc, b)
    matrix, indices = matrix_maker(fisher)
    print(matrix)
    sigma = find_sigma(matrix, indices)
    
    return fisher, derivatives, sigma, ss
    

In [25]:
def fisher_sim_qk(derivatives, parameters, step_sizes, vol_Gpc, num_gal, z, k_max):
    Pk, k_array, params = quick_Pk_zbins(vol_Gpc, z_bins, k_max)
    
    #simple so bias b = 1
    b = 1
    galaxy_density = num_gal/(vol_Gpc*(10**9))
    fisher = fisher_information(step_sizes, derivatives, Pk, k_array, z, galaxy_density, vol_Gpc, b)
    matrix, indices = matrix_maker(fisher)
    print(matrix)
    sigma = find_sigma(matrix, indices)
    
    return fisher, derivatives, sigma
    

In [26]:
def fisher_bias_qk(derivatives, parameters, step_sizes, vol_Gpc, num_gal, z, k_max, b):
    Pk, k_array, params = quick_Pk_zbins(vol_Gpc, z_bins, k_max)
    
    ss = copy.deepcopy(step_sizes)
    derivatives_b = add_bias_derivative(parameters, derivatives, b, vol_Gpc, z_bins, k_max)
    ss['b'] = 0
    
    galaxy_density = num_gal/(vol_Gpc*(10**9))
    fisher = fisher_information(ss, derivatives_b, Pk, k_array, z, galaxy_density, vol_Gpc, b)
    matrix, indices = matrix_maker(fisher)
    print(matrix)
    sigma = find_sigma(matrix, indices)
    
    return fisher, derivatives, sigma, ss
    

# Volume Methods

In [13]:
def H(a):
    return h*H100*np.sqrt(OmegaR*a**(-4) + OmegaM*a**(-3)+OmegaL)/hbar
def HMpc(a):
    return h*100*np.sqrt(OmegaR*a**(-4) + OmegaM*a**(-3)+OmegaL)  # returns H(a)  in km/s / Mpc

Spheredeg2=360**2/pi

def ztoa(z):
    return 1./(1.+z)

def distanceMpc(z):
    temp=quad(lambda x:1/(x*x*HMpc(x)),ztoa(z),1)[0]
    return temp*3*10.**(5.)*h
#return temp*3*10.**(5.)

def shell(z,Deltaz):
    temp=quad(lambda x:1/(x*x*HMpc(x)),ztoa(z+Deltaz),ztoa(z))[0]
    return temp*3*10.**(5.)*h
#return temp*3*10.**(5.)

def shellV(z,Deltaz,deg2):
    return 4*pi*shell(z,Deltaz)*distanceMpc(z)**2*deg2/Spheredeg2

def Dt(z):
    temp1=quad(lambda x:1/((x*HMpc(x)/100.)**3),0.0001,ztoa(z))[0]
    temp2=quad(lambda x:1/((x*HMpc(x)/100.)**3),0.0001,1)[0]
    return (temp1/temp2)*(HMpc(ztoa(z))/HMpc(1.))

def Dta(a):
    temp1=quad(lambda x:1/((x*HMpc(x)/100.)**3),0.0001,a)[0]
    temp2=quad(lambda x:1/((x*HMpc(x)/100.)**3),0.0001,1)[0]
    return (temp1/temp2)*(HMpc(a)/HMpc(1.))


def ng_z_bins(deg2,zmax,Nobs,z_space=0.1):
    z_bins=np.arange(z_space,zmax,z_space)
    ng_bins={}
    vol_tot=0
    for z in z_bins:
        vol_tot+=shellV(z,z_space,deg2)
    for z_j in z_bins:
        ng_bins[z_j]=Nobs/vol_tot
    return z_bins,ng_bins,vol_tot

Given a volume (calculated by some fraction of the sky and some range of redshifts), some average density of galaxies, and a range of redshifts, I can first make an approximation in which I will compute the fisher matrix as if it were a large rectangular box at the average redshift value. Or I can partition the volume into diffferent bins and individually compute their fisher matrices. Then I can compare my results and see how similar the numbers are.

In [14]:
def partition(deg2, z_min, z_max, dz, num_gal):
    Spheredeg2=360**2/pi
    frac_sky = deg2/Spheredeg2
    z_bins = np.arange(z_min, z_max + dz, dz) # +dz makes it inclusive
    vol_bins = np.zeros(len(z_bins))
    gal_bins = np.zeros(len(z_bins))

    vol = 0
    for z, i in zip(z_bins, range(len(z_bins))):
        vol_bins[i] = shellV(z, dz, deg2)/1e9
        vol += vol_bins[i]
        #Units of (Gpc/h)^3
    print(vol)
    gal_den = num_gal/vol
    
    #this calculates the number of galaxies needed to be in each bin, so that the galaxy densities are the
    #same in each bin (this is an approximation)
    for i in range(len(gal_bins)):
        gal_bins[i] = gal_den*vol_bins[i]

    return z_bins, vol_bins, gal_bins, vol

In [29]:
def fisher_many(parameters, step_sizes, z_bins, vol_bins, gal_bins, k_max, bias_bins):
    fisher_list = []
    for z, vol, num_gal, b in zip(z_bins, vol_bins, gal_bins, bias_bins):
        fisher, derivatives, sigma, ss = fisher_bias_all(parameters, step_sizes, vol, num_gal, z, k_max, b)
        fisher_list.append(fisher)
    size = len(fisher_list[0])
    matrix = np.zeros((size, size))
    for i in range(len(fisher_list)):
        matrix += matrix_maker(fisher_list[i])
    print(matrix)

In [30]:
step_sizes = {
    'tau_reio': 0.02,
    'n_s': 0.01,
    '100*theta_s': 0.002,
    'omega_b': 0.0008,
    'omega_cdm': 0.002
}
parameters = {
        'output': 'tCl pCl lCl mPk',
        'l_max_scalars': 5000,
        'lensing': 'yes',
        'n_s': 0.9667,
        '100*theta_s': 1.04112,
        'omega_b':0.02230,
        'omega_cdm': 0.1188,
        'tau_reio': 0.066,
}
k_max = 0.2

bias_bins = [1.79, 1.90, 1.98, 2.09, 2.32, 2.26, 2.38, 3.09]

deg2 = 10000
z_min = 0.05
z_max = 0.75
dz = 0.1
num_gal = 1.4e6

z_bins, vol_bins, gal_bins, vol_Gpc = partition(deg2, z_min, z_max, dz, num_gal)
fisher_many(parameters, step_sizes, z_bins, vol_bins, gal_bins, k_max, bias_bins)
fisher, derivatives, sigma, ss = fisher_bias_all(parameters, step_sizes, vol_Gpc, num_gal, z_bins[0], k_max, bias_bins[0])
print(fisher)

7.237301921031079
[[ 2.24312095e-03 -6.46989649e-01 -3.53999837e+00  8.67531151e+00
  -6.63151757e+00 -1.21375216e+00]
 [-6.46989649e-01  2.62921319e+02  8.76901570e+02 -3.38277395e+03
   2.51271517e+03  3.53175657e+02]
 [-3.53999837e+00  8.76901570e+02  8.62242748e+03 -1.03917938e+04
   6.39986830e+03  2.03252429e+03]
 [ 8.67531151e+00 -3.38277395e+03 -1.03917938e+04  5.04105300e+04
  -3.80919833e+04 -5.06904248e+03]
 [-6.63151757e+00  2.51271517e+03  6.39986830e+03 -3.80919833e+04
   3.00415521e+04  3.77249027e+03]
 [-1.21375216e+00  3.53175657e+02  2.03252429e+03 -5.06904248e+03
   3.77249027e+03  6.95107433e+02]]
Determinant : 17872797.383502685
Error for tau_reio is : 183.04777740285311
Error for n_s is : 1.8195127679950693
Error for 100*theta_s is : 0.8106481658802216
Error for omega_b is : 0.17556394986469806
Error for omega_cdm is : 0.690634824942326
Error for b is : 4.01778185312588
[[ 1.58287672e-02 -5.02913465e+00 -3.11198297e+01  5.80067636e+01
  -4.36026757e+01 -8.76739729

KeyboardInterrupt: 

## Initial parameters we care about:

In [ ]:
step_sizes = {
    'A_s': 1e-10,
    'N_ur': 0.06,
    'tau_reio': 0.02,
    'n_s': 0.01,
    '100*theta_s': 0.002,
    'omega_b': 0.0008,
    'omega_cdm': 0.002
}
parameters = {
        'output': 'tCl pCl lCl mPk',
        'l_max_scalars': 5000,
        'lensing': 'yes',
        'A_s': 2.1e-9,
        'n_s': 0.9667,
        '100*theta_s': 1.04112,
        'omega_b':0.02230,
        'omega_cdm': 0.1188,
        'tau_reio': 0.066,
        'N_ur': 3.046
}
vol_Gpc = 200
num_gal = 2e9
z_bins = [0] #redshift values
k_max = 0.7
z = 0 #choosing redshift of zero
galaxy_density = 1e-2 #number of galaxies in a Megaparsec^3

In [ ]:
# fisher, derivatives, sigma = fisher_sim_all(parameters, step_sizes, vol_Gpc, num_gal, z_bins, k_max)
# filename = 'dfile'
# outfile = open(filename, 'wb')
# pickle.dump(derivatives, outfile)
# outfile.close()

In [ ]:
filename = 'dfile'
infile = open(filename, 'rb')
derivatives = pickle.load(infile)
infile.close()
fisher, derivatives, sigma = fisher_sim_qk(derivatives, parameters, step_sizes, vol_Gpc, num_gal, z_bins, k_max)

Note that in the following cell, I am using a different list for the step sizes where I have removed A_s because it is degenerate with the bias b. If I were to keep it, one of the eigenvalues would be zero, but with the computer precision, it was setting it to a very small negative number. This negative number then made the determinant negative, so when I tried to take the square root of the diagonal entries of the inverted matrix, it gave me imaginary numbers.

In [ ]:
step_sizes_b = {
    'N_ur': 0.06,
    'tau_reio': 0.02,
    'n_s': 0.01,
    '100*theta_s': 0.002,
    'omega_b': 0.0008,
    'omega_cdm': 0.002
}

b = 1.79

In [ ]:
# fisher_b, derivatives_no_A_s, sigma_b, step_sizes_bias = fisher_bias_all(parameters, step_sizes_b, vol_Gpc, num_gal, z_bins, k_max, b)
# filename = 'dfile_no_A_s'
# outfile = open(filename, 'wb')
# pickle.dump(derivatives_no_A_s, outfile)
# outfile.close()


In [ ]:
filename = 'dfile_no_A_s'
infile = open(filename, 'rb')
derivatives_no_A_s = pickle.load(infile)
infile.close()
fisher_b, derivatives_no_A_s, sigma_b, step_sizes_bias = fisher_bias_qk(derivatives_no_A_s, parameters, step_sizes_b, vol_Gpc, num_gal, z_bins, k_max, b)